In [1]:
import sys
import re
import numpy as np
from numpy import dot
from numpy.linalg import norm
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.ml.linalg import Vectors, VectorUDT
from operator import add

In [2]:
File = 'Books_5.json'
df = spark.read.json(File)

In [3]:
df.count()

8898041

In [3]:
df_full = df.drop('asin').drop('helpful').drop('reviewTime').\
                drop('reviewerID').drop('reviewerName').drop('summary').drop('unixReviewTime')

In [4]:
pn_udf = udf(lambda x: 1 if x >=4 else 0, IntegerType())
df_full_l = df_full.withColumn('label',pn_udf('overall')).drop('overall')

In [5]:
df_full_l.show()

+--------------------+-----+
|          reviewText|label|
+--------------------+-----+
|Spiritually and m...|    1|
|This is one my mu...|    1|
|This book provide...|    1|
|I first read THE ...|    1|
|A timeless classi...|    1|
|Reading this made...|    1|
|As you read, Gibr...|    1|
|Deep, moving dram...|    1|
|This is a timeles...|    1|
|An amazing work. ...|    1|
|I LOVE this book....|    1|
|This book has bee...|    1|
|This book has so ...|    1|
|When I was in col...|    1|
|I discovered The ...|    1|
|Can't say enough ...|    1|
|I have the 1972 v...|    1|
|Anything I've rea...|    1|
|Cool book, I real...|    1|
|This book is ever...|    1|
+--------------------+-----+
only showing top 20 rows



In [6]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer
dim = 20000
tokenizer = Tokenizer(inputCol = 'reviewText', outputCol = 'words')
wordsData = tokenizer.transform(df_full_l)

In [7]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=dim)
featurizedData = hashingTF.transform(wordsData)

In [8]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

In [9]:
rescaledData = idfModel.transform(featurizedData)
tfifd = rescaledData.select("label", "features")

In [10]:
df_train, df_test = tfifd.randomSplit([.7, .3])

In [11]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=100)
model = lr.fit(df_train)
predictions = model.transform(df_test)

In [12]:
result = predictions.select('label','prediction').collect()
y = []
y_pred = []

for i in result:
    y.append(i[0])
    y_pred.append(i[1])

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
conf = confusion_matrix(y, y_pred)
print('confusion matrix is ')
print(conf)
print('accuracy is:')
print(accuracy_score(y, y_pred))
print('f1 score is:')
print(f1_score(y, y_pred))

confusion matrix is 
[[ 252983  255258]
 [  71518 2087564]]
accuracy is:
0.8774891529822223
f1 score is:
0.9274138231290583
